# Preprocess chain from Raster/SHP to input format of the net

Requirements can be found in requirements.txt
This code relies on GDAL


Set the directory path for the project

In [1]:
#make dir where the root folder of the project is
%env DIR=/eos/jeodpp/data/projects/REFOCUS/data/swalim_v2
!pwd


env: DIR=/eos/jeodpp/data/projects/REFOCUS/data/swalim_v2
/eos/jeodpp/data/projects/REFOCUS/data/swalim_v2/code


In [ ]:
# remove BROWSE files if they exist
find . -name '*BROWSE*' -delete

Create the tiles to run the model
The output images goes to inputs/Tiled
Variables to set:
* SIZE  size in pixels of the tile cut
* JOBS number of parallel jobs desired to run this code
* OVERLAP number of pixels you want the tiles to overlap. Useful to later handle better the objects detected on the edges

In [16]:
%%bash
SIZE=1024
JOBS=36
OVERLAP=20
pwd
mkdir -p $DIR/scripts_toerase/
cd $DIR/code/scripts/processing/
mkdir -p $DIR/inputs/Tiled
chmod +x *.sh
for file in $DIR/images/*/*tif
do
    echo $DIR/code/scripts/processing/maketiles.sh ${file} $SIZE $(basename "$file" .tif) 20 $DIR/inputs/Tiled/ > $DIR/scripts_toerase/'mt_'$(basename "$file")'.sh'
done


/eos/jeodpp/data/projects/REFOCUS/data/swalim_v2/code


chmod: changing permissions of 'maketiles.sh': Operation not permitted
chmod: changing permissions of 'merge-shp.sh': Operation not permitted


In [17]:
%%bash
chmod +x $DIR/scripts_toerase/mt_*.sh
ls $DIR/scripts_toerase/mt_* > $DIR/scripts_toerase/Main-par.txt
ls $DIR/scripts_toerase/mt_* | wc -l
cat $DIR/scripts_toerase/Main-par.txt | /eos/jeodpp/data/projects/REFOCUS/data/swalim_v2/code/parallel/bin/parallel -j 36 {}
rm -r  $DIR/scripts_toerase

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
%%bash
pwd
find $DIR/inputs/Tiled/ -type f -maxdepth 1 -name "*tif"> $DIR/inputs/list_tiles.csv
mkdir -p $DIR/inputs/Tiled/pancro/ $DIR/inputs/Tiled/RGB/

/eos/jeodpp/data/projects/REFOCUS/data/swalim_v2/code


### Erase tiles that are all 0
Also separate tiles that belong to RGB and Pancro in diferent folders
in this case it creates a compilation of directories of 100 images under each type. To ensure computation


In [20]:
%%bash
cd $DIR/code/scripts/processing/
pwd
python3 erase_tiles_nodata.py

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
%%bash
# CLEAN THE tILES FOLDERS FROM *XML AND *tfw
rm $DIR/inputs/Tiled/*xml
rm $DIR/inputs/Tiled/*tfw
# Create subfolders of 100 elements each to be able to handle them
cd $DIR/inputs/Tiled/pancro
i=0; for f in *; do d=dir_$(printf %03d $((i/100+1))); mkdir -p $d; mv "$f" $d; let i++; done
cd $DIR/inputs/Tiled/RGB
i=0; for f in *; do d=dir_$(printf %03d $((i/100+1))); mkdir -p $d; mv "$f" $d; let i++; done


bash: line 2: /bin/rm: Argument list too long
bash: line 3: /bin/rm: Argument list too long


Create the footprints of the tiles, one for RBG and another one for the pancromatic images

In [26]:
%%bash
rm -r $DIR/inputs/footprints
mkdir -p $DIR/inputs/footprints

#create the footprints for RGB and pancro images
find $DIR/inputs/Tiled/pancro/dir_* -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  $DIR/inputs/footprints/footprint-pancro.shp 
find $DIR/inputs/Tiled/RGB/dir_* -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  $DIR/inputs/footprints/footprint-RGB.shp

find $DIR/inputs/Tiled/pancro/dir_* -type f -maxdepth 1 -name "*tif" > $DIR/inputs/list_tiles_pancro.csv
find $DIR/inputs/Tiled/RGB/dir_* -type f -maxdepth 1 -name "*tif" > $DIR/inputs/list_tiles_RGB.csv

Creating new index file...
Creating new index file...


### Run the preprocess algorithm to create the dataset to train/validate/test

In this case we will need the shapefile where teh ground truth data is delineated, in our case is in DIR/extended_AOI/
Parameters:
- CSV with the list of images
- Where to output the image. The code handles the distirbution of pancro and RGB
- Folder to put in images with annotations
- Folder to put in images without annotations
- The name of the shapefile with the objects to train
- Name and path of the json file where the annotations will be saved

In [35]:
%%bash
pwd
cd $DIR/code/scripts/processing/
mkdir -p  $DIR/inputs/pancro/img_with_ann/  $DIR/inputs/RGB/img_with_ann/ $DIR/inputs/pancro/img_without_ann/  $DIR/inputs/RGB/img_without_ann/ $DIR/inputs/pancro/annotations/  $DIR/inputs/RGB/annotations/
python3 image_preprocess.py $DIR/inputs/list_tiles_pancro.csv $DIR/inputs/pancro/img_with_ann/ $DIR/inputs/pancro/img_without_ann/ $DIR/extended_AOI/2017_2019_extendedAOI_Feb2020.shp $DIR/inputs/pancro/annotations/pancro_all.json 
python3 image_preprocess.py $DIR/inputs/list_tiles_RGB.csv $DIR/inputs/RGB/img_with_ann/ $DIR/inputs/RGB/img_without_ann/ $DIR/extended_AOI/2017_2019_extendedAOI_Feb2020.shp $DIR/inputs/RGB/annotations/RGB_all.json

/eos/jeodpp/data/projects/REFOCUS/data/swalim_v2/code
Finish!!! :). Execution time: 30838.1694419384
Finish!!! :). Execution time: 90028.31702136993


Create the new footprints, one for images with annotatins and another one for images without

In [36]:
%%bash
# pancro images
path_pancro_ann=$DIR/inputs/pancro/img_with_ann/
path_pancro_noann=$DIR/inputs/pancro/img_without_ann/ 

find $path_pancro_noann -type f -maxdepth 1 -name "*tif" > $DIR/inputs/pancro_listwithoutann.csv
find $path_pancro_ann  -type f -maxdepth 1 -name "*tif" > $DIR/inputs/pancro_listwithann.csv

find $path_pancro_noann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  $DIR/inputs/footprints/footprint-pancro-noann.shp 
find $path_pancro_ann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  $DIR/inputs/footprints/footprint-pancro-ann.shp

# RGB images
path_RGB_ann=$DIR/inputs/RGB/img_with_ann/
path_RGB_noann=$DIR/inputs/RGB/img_without_ann/ 

find $path_RGB_noann -type f -maxdepth 1 -name "*tif" > $DIR/inputs/RGB_listwithoutann.csv
find $path_RGB_ann  -type f -maxdepth 1 -name "*tif" > $DIR/inputs/RGB_listwithann.csv

find $path_RGB_noann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  $DIR/inputs/footprints/footprint-RGB-noann.shp 
find $path_RGB_ann -name '*.tif' -print | xargs --max-args=50 gdaltindex -t_srs EPSG:4326 -src_srs_name src_srs  $DIR/inputs/footprints/footprint-RGB-ann.shp

Creating new index file...
Creating new index file...
Creating new index file...
Creating new index file...


In [46]:
%%bash
#change paths inside the files
sed -i "s|/scratch/swalim/inputs/RGB/img_with_ann|$DIR/inputs/RGB/img_with_ann|g" $DIR/inputs/RGB_listwithann.csv
sed -i "s|/scratch/swalim/inputs/pancro/img_with_ann|$DIR/inputs/pancro/img_with_ann|g" $DIR/inputs/pancro_listwithann.csv

sed -i "s|/scratch/swalim/inputs/RGB/img_without_ann|$DIR/inputs/RGB/img_without_ann|g" $DIR/inputs/RGB_listwithoutann.csv
sed -i "s|/scratch/swalim/inputs/pancro/img_without_ann|$DIR/inputs/pancro/img_without_ann|g" $DIR/inputs/pancro_listwithoutann.csv

